In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os, shutil
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Force offline behavior for Hugging Face
os.environ.setdefault("HF_HUB_OFFLINE", "1")
os.environ.setdefault("TRANSFORMERS_OFFLINE", "1")

# Avoid importing TF/JAX backends via transformers and quiet TF/XLA logs
os.environ.setdefault("TRANSFORMERS_NO_TF", "1")
os.environ.setdefault("TRANSFORMERS_NO_FLAX", "1")
os.environ.setdefault("TF_CPP_MIN_LOG_LEVEL", "3")
os.environ.setdefault("TOKENIZERS_PARALLELISM", "false")

/kaggle/input/embeddings/other/default/1/train_embeddings_a.npy
/kaggle/input/embeddings/other/default/1/train_embeddings_b.npy
/kaggle/input/embeddings/other/default/1/test_embeddings_a.npy
/kaggle/input/embeddings/other/default/1/test_embeddings_b.npy
/kaggle/input/all-minilm-l6-v2/transformers/default/1/all-MiniLM-L6-v2/rust_model.ot
/kaggle/input/all-minilm-l6-v2/transformers/default/1/all-MiniLM-L6-v2/config.json
/kaggle/input/all-minilm-l6-v2/transformers/default/1/all-MiniLM-L6-v2/README.md
/kaggle/input/all-minilm-l6-v2/transformers/default/1/all-MiniLM-L6-v2/tokenizer.json
/kaggle/input/all-minilm-l6-v2/transformers/default/1/all-MiniLM-L6-v2/tf_model.h5
/kaggle/input/all-minilm-l6-v2/transformers/default/1/all-MiniLM-L6-v2/data_config.json
/kaggle/input/all-minilm-l6-v2/transformers/default/1/all-MiniLM-L6-v2/train_script.py
/kaggle/input/all-minilm-l6-v2/transformers/default/1/all-MiniLM-L6-v2/tokenizer_config.json
/kaggle/input/all-minilm-l6-v2/transformers/default/1/all-Mi

'false'

# 0) CONFIG

In [ ]:
# 0) CONFIG
MODE = "ensemble"  # "lexical" | "embeddings" | "embeddings_lora" | "ensemble"
N_SPLITS = 5
SEED = 42

# Kaggle data
TRAIN_PATH = os.environ.get("TRAIN_PATH", "/kaggle/input/llm-classification-finetuning/train.csv")
TEST_PATH  = os.environ.get("TEST_PATH",  "/kaggle/input/llm-classification-finetuning/test.csv")

# Sentence-Transformers local folder (attach a dataset and point here)
ST_EMB_PATH = "/kaggle/input/all-minilm-l6-v2/transformers/default/1/all-MiniLM-L6-v2"

# ---- Embedding assets ----
EMBED_A_TRAIN_IN = "/kaggle/input/final-embeddings/train_embeddings_a.npy"
EMBED_B_TRAIN_IN = "/kaggle/input/final-embeddings/train_embeddings_b.npy"
EMBED_A_TEST_IN  = "/kaggle/input/final-embeddings/test_embeddings_a.npy"
EMBED_B_TEST_IN  = "/kaggle/input/final-embeddings/test_embeddings_b.npy"

TRAIN_A_PATH = os.environ.get("TRAIN_A_PATH", "/kaggle/working/train_embeddings_a.npy")
TRAIN_B_PATH = os.environ.get("TRAIN_B_PATH", "/kaggle/working/train_embeddings_b.npy")
TEST_A_PATH  = os.environ.get("TEST_A_PATH",  "/kaggle/working/test_embeddings_a.npy")
TEST_B_PATH  = os.environ.get("TEST_B_PATH",  "/kaggle/working/test_embeddings_b.npy")

# LoRA backbone (must be a local folder in /kaggle/input)
LORA_MODEL_PATH = os.environ.get("LORA_MODEL_PATH", "/kaggle/input/distil_bert/keras/distil_bert_base_en_uncased/3")

# 1) IMPORTS & SEEDING

In [3]:
import re, ast, random, warnings
import numpy as np, pandas as pd
warnings.filterwarnings('ignore')


from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics import log_loss


random.seed(SEED)
np.random.seed(SEED)

# 2) DATA LOADING & BASIC TEXT EXTRACTION

In [4]:
train_df = pd.read_csv(TRAIN_PATH).dropna()
test_df = pd.read_csv(TEST_PATH).dropna()


# Build 3-class target: 0=A, 1=B, 2=Tie
y = np.select([
train_df['winner_model_a'] == 1,
train_df['winner_model_b'] == 1,
train_df['winner_tie'] == 1
], [0, 1, 2])
classes = [0, 1, 2]


# Text fields may be JSON-ish lists; normalize to strings
def extract_text_from_field(text_field):
    try:
        parsed = ast.literal_eval(text_field)
        if isinstance(parsed, list):
            return ' '.join(map(str, parsed))
        return str(parsed)
    except Exception:
        return str(text_field)


for df in (train_df, test_df):
    df['prompt_text'] = df['prompt'].apply(extract_text_from_field)
    df['response_a_text'] = df['response_a'].apply(extract_text_from_field)
    df['response_b_text'] = df['response_b'].apply(extract_text_from_field)
    df['text_a'] = df['prompt_text'] + ' [SEP] ' + df['response_a_text']
    df['text_b'] = df['prompt_text'] + ' [SEP] ' + df['response_b_text']

# 3) SHARED HELPERS (Folds, CV, Inference, Utils)

In [5]:
skf = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=SEED)
fold_indices = [(tr, va) for tr, va in skf.split(np.zeros(len(y)), y)]

def cv_calibrated_probs_numpy(X, y, method='sigmoid'):
    """Cross-validated calibrated probabilities for numpy arrays.
       Returns OOF probs + list of fold models & scalers.
    """
    assert len(X) == len(y), f"X rows {len(X)} must equal y {len(y)}"
    oof = np.zeros((len(X), 3), dtype=float)
    models, scalers = [], []
    for fold, (tr, va) in enumerate(fold_indices, 1):
        scaler = StandardScaler()
        Xtr_s = scaler.fit_transform(X[tr])
        Xva_s = scaler.transform(X[va])

        base = LogisticRegression(max_iter=2000, C=1.0, random_state=SEED)
        clf  = CalibratedClassifierCV(estimator=base, method=method, cv=3)
        clf.fit(Xtr_s, y[tr])
        proba = clf.predict_proba(Xva_s)
        oof[va] = proba
        print(f"[CV {method}] fold {fold} logloss = {log_loss(y[va], proba, labels=classes):.5f}")
        models.append(clf); scalers.append(scaler)
    print(f"[CV {method}] OOF logloss = {log_loss(y, oof, labels=classes):.5f}")
    return oof, models, scalers

def cv_calibrated_probs_df(X_df, y, method='sigmoid'):
    """Same as above but accepts a pandas DataFrame (for lexical features)."""
    X = X_df.values.astype(float)
    return cv_calibrated_probs_numpy(X, y, method=method)

def predict_from_folds(models, scalers, X):
    P = np.zeros((len(X), 3), dtype=float)
    for clf, scaler in zip(models, scalers):
        Xt = scaler.transform(X)
        P += clf.predict_proba(Xt) / len(models)
    return P

# Simple 1-D weight search for 2-model blend
def best_weight_for_blend(y, p1, p2, steps=101):
    best_w, best_loss = 0.5, 1e9
    for i in range(steps):
        w = i/(steps-1)
        blend = w*p1 + (1-w)*p2
        loss  = log_loss(y, blend, labels=classes)
        if loss < best_loss:
            best_loss, best_w = loss, w
    return best_w, best_loss

# Optional temperature scaling (post-hoc smoothing on probs)
def fit_temperature(p_oof, y, iters=50):
    z = np.log(np.clip(p_oof, 1e-8, 1.0)).astype(float)
    z = z - z.mean(axis=1, keepdims=True)
    T, best = 1.0, 1e9
    for _ in range(iters):
        for t in np.linspace(max(0.5, T-0.5), T+0.5, 9):
            q = np.exp(z/t); q /= q.sum(axis=1, keepdims=True)
            ll = log_loss(y, q, labels=classes)
            if ll < best:
                best, T = ll, t
    return T

def apply_temperature(p, T):
    z = np.log(np.clip(p, 1e-8, 1.0)).astype(float)
    z = z - z.mean(axis=1, keepdims=True)
    q = np.exp(z/T); q /= q.sum(axis=1, keepdims=True)
    return q

# 4) LEXICAL FEATURES (bias-aware verbosity/structure)

In [6]:
import math

def count_pattern(text, pattern):
    if not isinstance(text, str):
        return 0
    return len(re.findall(pattern, text, flags=re.MULTILINE))

def paragraph_count(t):
    return t.count('\n\n') if isinstance(t, str) else 0

def list_count(t):
    return count_pattern(t, r"^\s*(?:[\-\*•]\s|\d+\.)")

def quote_count(t):
    # '>' line quotes or **bold** emphasis as a proxy
    return count_pattern(t, r"(^>\s|(?<!\*)\*\*[^*]+\*\*)")

def sentence_count(t):
    return count_pattern(t, r"[.!?](?:\s|$)")

def code_block_count(t):
    return count_pattern(t, r"```|`[^`]+`")

def heading_count(t):
    return count_pattern(t, r"^(?:#{1,6})\s")

def word_count(t):
    return len(t.split()) if isinstance(t, str) else 0

# Build a rich lexical matrix for A/B and contrasts
def build_lex_features(df, a_col='response_a_text', b_col='response_b_text'):
    A = df[a_col].fillna('').astype(str)
    B = df[b_col].fillna('').astype(str)
    feats = {}
    for name, series in [('a', A), ('b', B)]:
        feats[f'len_{name}']   = series.map(len)
        feats[f'wc_{name}']    = series.map(word_count)
        feats[f'sent_{name}']  = series.map(sentence_count)
        feats[f'para_{name}']  = series.map(paragraph_count)
        feats[f'list_{name}']  = series.map(list_count)
        feats[f'quote_{name}'] = series.map(quote_count)
        feats[f'code_{name}']  = series.map(code_block_count)
        feats[f'hdr_{name}']   = series.map(heading_count)
    F = pd.DataFrame(feats).astype(float)
    # Diffs & ratios
    for base in ['len','wc','sent','para','list','quote','code','hdr']:
        F[f'{base}_diff']  = F[f'{base}_a'] - F[f'{base}_b']
    for base in ['len','wc','sent']:
        F[f'{base}_ratio'] = (F[f'{base}_a'] + 1.0) / (F[f'{base}_b'] + 1.0)
    # Densities per 100 words
    eps = 1e-6
    for base in ['sent','para','list','quote','code','hdr']:
        F[f'{base}_per100w_a'] = 100.0 * F[f'{base}_a'] / (F['wc_a'] + eps)
        F[f'{base}_per100w_b'] = 100.0 * F[f'{base}_b'] / (F['wc_b'] + eps)
        F[f'{base}_per100w_diff'] = F[f'{base}_per100w_a'] - F[f'{base}_per100w_b']
    F = F.replace([np.inf, -np.inf], 0.0).fillna(0.0)
    return F

lex_cols_keep = [
    # core diffs (matches the original minimal set)
    'len_diff','wc_diff','sent_diff','para_diff','list_diff','quote_diff','code_diff','hdr_diff',
    'len_ratio','wc_ratio','sent_ratio'
]

X_lex_train_df = build_lex_features(train_df)
X_lex_test_df  = build_lex_features(test_df)
# Keep a compact set to remain fully coherent with the original
X_lex = X_lex_train_df[lex_cols_keep].copy()
X_lex_test = X_lex_test_df[lex_cols_keep].copy()
print("X_lex:", X_lex.shape, " X_lex_test:", X_lex_test.shape)

X_lex: (57477, 11)  X_lex_test: (3, 11)


# 5) LEXICAL MODEL (run if MODE in {lexical, ensemble})

In [7]:
lex_best_name = None
lex_best_oof = None
lex_best_models = []
lex_best_scalers = []

if MODE in {"lexical", "ensemble"}:
    oof_lex_sig, lex_sig_models, lex_sig_scalers = cv_calibrated_probs_df(X_lex, y, method='sigmoid')
    oof_lex_iso, lex_iso_models, lex_iso_scalers = cv_calibrated_probs_df(X_lex, y, method='isotonic')

    if log_loss(y, oof_lex_iso, labels=classes) < log_loss(y, oof_lex_sig, labels=classes):
        lex_best_name   = 'isotonic'
        lex_best_oof    = oof_lex_iso
        lex_best_models = lex_iso_models
        lex_best_scalers= lex_iso_scalers
    else:
        lex_best_name   = 'sigmoid'
        lex_best_oof    = oof_lex_sig
        lex_best_models = lex_sig_models
        lex_best_scalers= lex_sig_scalers

    # TEST inference
    Xlt = X_lex_test.values.astype(float)
    lex_proba_test = predict_from_folds(lex_best_models, lex_best_scalers, Xlt)
    sub_lex = pd.DataFrame({
    'id': test_df['id'].values,
    'winner_model_a': lex_proba_test[:,0],
    'winner_model_b': lex_proba_test[:,1],
    'winner_tie':     lex_proba_test[:,2],
    })


[CV sigmoid] fold 1 logloss = 1.07003
[CV sigmoid] fold 2 logloss = 1.06940
[CV sigmoid] fold 3 logloss = 1.07065
[CV sigmoid] fold 4 logloss = 1.07088
[CV sigmoid] fold 5 logloss = 1.06935
[CV sigmoid] OOF logloss = 1.07006
[CV isotonic] fold 1 logloss = 1.06102
[CV isotonic] fold 2 logloss = 1.06256
[CV isotonic] fold 3 logloss = 1.06173
[CV isotonic] fold 4 logloss = 1.06018
[CV isotonic] fold 5 logloss = 1.06073
[CV isotonic] OOF logloss = 1.06124


# 6) EMBEDDINGS (run if MODE in {embeddings, ensemble})

In [8]:
# --- SINGLE embeddings block (keep only this one) ---
X_emb_train = None
X_emb_test  = None
emb_best_name = None
emb_best_oof = None
emb_best_models = []
emb_best_scalers = []

if MODE in {"embeddings", "ensemble"}:
    import ast, os, torch
    from sentence_transformers import SentenceTransformer
    from transformers import AutoTokenizer

    def _all_exist(paths):
        return all(isinstance(p, str) and p != "NONE" and os.path.exists(p) for p in paths)

    # 1) Try input-dataset .npy
    if _all_exist([EMBED_A_TRAIN_IN, EMBED_B_TRAIN_IN, EMBED_A_TEST_IN, EMBED_B_TEST_IN]):
        print("✅ Loading embeddings from INPUT DATASET paths...")
        train_a = np.load(EMBED_A_TRAIN_IN); train_b = np.load(EMBED_B_TRAIN_IN)
        test_a  = np.load(EMBED_A_TEST_IN);  test_b  = np.load(EMBED_B_TEST_IN)
        # also save to working dir so future runs detect them
        np.save(TRAIN_A_PATH, train_a); np.save(TRAIN_B_PATH, train_b)
        np.save(TEST_A_PATH,  test_a);  np.save(TEST_B_PATH,  test_b)

    # 2) Else working-dir cache
    elif _all_exist([TRAIN_A_PATH, TRAIN_B_PATH, TEST_A_PATH, TEST_B_PATH]):
        print("✅ Loading embeddings from WORKING DIR cache...")
        train_a = np.load(TRAIN_A_PATH); train_b = np.load(TRAIN_B_PATH)
        test_a  = np.load(TEST_A_PATH);  test_b  = np.load(TEST_B_PATH)

    # 3) Else compute (and save to working dir)
    else:
        print("⚠️ No saved embeddings found → computing now...")

        def extract_text_from_field(v):
            if v is None or (isinstance(v, float) and pd.isna(v)): return ""
            try:
                parsed = ast.literal_eval(str(v))
                if isinstance(parsed, list): return " ".join(str(x) for x in parsed)
                return str(parsed)
            except Exception:
                return str(v)

        for df in (train_df, test_df):
            df["prompt_text"]     = df["prompt"].apply(extract_text_from_field)
            df["response_a_text"] = df["response_a"].apply(extract_text_from_field)
            df["response_b_text"] = df["response_b"].apply(extract_text_from_field)
            df["text_a"] = df["prompt_text"] + " [SEP] " + df["response_a_text"]
            df["text_b"] = df["prompt_text"] + " [SEP] " + df["response_b_text"]

        train_text_a = train_df["text_a"].tolist()
        train_text_b = train_df["text_b"].tolist()
        test_text_a  = test_df["text_a"].tolist()
        test_text_b  = test_df["text_b"].tolist()

        print("\nLoading SentenceTransformer model from:", ST_EMB_PATH)
        device = "cuda" if torch.cuda.is_available() else "cpu"
        model = SentenceTransformer(ST_EMB_PATH, device=device)
        print(f"→ Running embeddings on device: {device}")

        slow_tok = AutoTokenizer.from_pretrained(ST_EMB_PATH, use_fast=False, local_files_only=True)
        model._first_module().tokenizer = slow_tok
        model.max_seq_length = 512

        BATCH_SIZE = 32
        print("\nEncoding training embeddings A...")
        train_a = model.encode(train_text_a, batch_size=BATCH_SIZE, show_progress_bar=True, convert_to_numpy=True)
        print("\nEncoding training embeddings B...")
        train_b = model.encode(train_text_b, batch_size=BATCH_SIZE, show_progress_bar=True, convert_to_numpy=True)
        print("\nEncoding test embeddings A...")
        test_a = model.encode(test_text_a, batch_size=BATCH_SIZE, show_progress_bar=True, convert_to_numpy=True)
        print("\nEncoding test embeddings B...")
        test_b = model.encode(test_text_b, batch_size=BATCH_SIZE, show_progress_bar=True, convert_to_numpy=True)

        np.save(TRAIN_A_PATH, train_a); np.save(TRAIN_B_PATH, train_b)
        np.save(TEST_A_PATH,  test_a);  np.save(TEST_B_PATH,  test_b)
        print(f"💾 Saved embeddings to:\n  {TRAIN_A_PATH}\n  {TRAIN_B_PATH}\n  {TEST_A_PATH}\n  {TEST_B_PATH}")

    # Build, calibrate, submit
    X_emb_train = np.concatenate([train_a, train_b], axis=1)
    X_emb_test  = np.concatenate([test_a,  test_b],  axis=1)
    print("Embedding shapes:", X_emb_train.shape, X_emb_test.shape)

    oof_emb_sig, emb_sig_models, emb_sig_scalers = cv_calibrated_probs_numpy(X_emb_train, y, method="sigmoid")
    oof_emb_iso, emb_iso_models, emb_iso_scalers = cv_calibrated_probs_numpy(X_emb_train, y, method="isotonic")

    if log_loss(y, oof_emb_iso, labels=classes) < log_loss(y, oof_emb_sig, labels=classes):
        emb_best_name, emb_best_oof = "isotonic", oof_emb_iso
        emb_best_models, emb_best_scalers = emb_iso_models, emb_iso_scalers
    else:
        emb_best_name, emb_best_oof = "sigmoid", oof_emb_sig
        emb_best_models, emb_best_scalers = emb_sig_models, emb_sig_scalers

    print(f"\n✅ Best embedding calibration: {emb_best_name}")

    emb_proba_test = predict_from_folds(emb_best_models, emb_best_scalers, X_emb_test)
else:
    print("MODE does not include embeddings → skipping.")
    emb_proba_test = None

2025-11-05 22:28:24.527325: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1762381704.551609      92 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1762381704.558740      92 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


✅ Loading embeddings from INPUT DATASET paths...
Embedding shapes: (57477, 768) (3, 768)
[CV sigmoid] fold 1 logloss = 1.05724
[CV sigmoid] fold 2 logloss = 1.06211
[CV sigmoid] fold 3 logloss = 1.05590
[CV sigmoid] fold 4 logloss = 1.05847
[CV sigmoid] fold 5 logloss = 1.05856
[CV sigmoid] OOF logloss = 1.05846
[CV isotonic] fold 1 logloss = 1.05629
[CV isotonic] fold 2 logloss = 1.06120
[CV isotonic] fold 3 logloss = 1.05486
[CV isotonic] fold 4 logloss = 1.05782
[CV isotonic] fold 5 logloss = 1.06042
[CV isotonic] OOF logloss = 1.05812

✅ Best embedding calibration: isotonic


# 7) LoRA

In [ ]:
# 7) EMBEDDINGS_LORA (run if MODE in {embeddings_lora, ensemble})
lora_test_proba = None
lora_oof = None

if MODE == "embeddings_lora":
    import os, torch
    from datasets import Dataset
    from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
    from peft import LoraConfig, get_peft_model

    if not os.path.isdir(LORA_MODEL_PATH):
        raise RuntimeError(
            f"LoRA MODE selected but local model folder not found at '{LORA_MODEL_PATH}'. "
            "Attach a model to the notebook and set LORA_MODEL_PATH accordingly."
        )

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # Build compact triplets from normalized text columns (already created earlier)
    def build_input(df):
        # short tags reduce sequence length vs full [SEP] variants
        return (
            '[PROMPT] ' + df['prompt_text'] +
            ' [A] ' + df['response_a_text'] +
            ' [B] ' + df['response_b_text']
        )

    train_inputs = build_input(train_df)
    test_inputs  = build_input(test_df)

    # Tokenizer & datasets (offline)
    # local_files_only=True + HF offline env ensures no internet hit
    tokenizer = AutoTokenizer.from_pretrained(LORA_MODEL_PATH, use_fast=True, local_files_only=True)

    MAX_LEN = 256  # faster, usually no accuracy drop for this task
    def tokenize_fn(batch):
        return tokenizer(batch['text'], truncation=True, padding='max_length', max_length=MAX_LEN)

    ds_train = Dataset.from_pandas(pd.DataFrame({'text': train_inputs, 'label': y}))
    ds_test  = Dataset.from_pandas(pd.DataFrame({'text': test_inputs}))
    tokenized_train = ds_train.map(tokenize_fn, batched=True, remove_columns=['text'])
    tokenized_test  = ds_test.map(tokenize_fn, batched=True, remove_columns=['text'])

    # Small train/val split to produce pseudo-OOF for ensembling/calibration
    tr_idx, va_idx = train_test_split(np.arange(len(ds_train)), test_size=0.15, random_state=SEED, stratify=y)
    ds_tr = tokenized_train.select(tr_idx.tolist())
    ds_va = tokenized_train.select(va_idx.tolist())

    # Model (offline) + LoRA adapters
    torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32
    model = AutoModelForSequenceClassification.from_pretrained(
        LORA_MODEL_PATH, num_labels=3, torch_dtype=torch_dtype, local_files_only=True
    )

    # Choose reasonable target modules for LoRA depending on backbone
    # DistilBERT uses q_lin/v_lin; BERT/Roberta use query/key/value; DeBERTa uses *_proj
    all_module_names = [n for n,_ in model.named_modules()]
    if any("q_lin" in n for n in all_module_names) or any("v_lin" in n for n in all_module_names):
        target_modules = ["q_lin","v_lin"]
    elif any(".query" in n for n in all_module_names):
        target_modules = ["query","key","value","dense"]
    elif any("q_proj" in n for n in all_module_names):
        target_modules = ["q_proj","k_proj","v_proj","dense"]
    else:
        # generic fallback
        target_modules = ["query","key","value","dense","q_proj","k_proj","v_proj","q_lin","v_lin"]

    peft_config = LoraConfig(r=8, lora_alpha=16, lora_dropout=0.05, task_type='SEQ_CLS', target_modules=target_modules)
    model = get_peft_model(model, peft_config).to(device)

    def compute_metrics(eval_pred):
        logits, labels = eval_pred
        probs = torch.softmax(torch.tensor(logits), dim=1).numpy()
        return {'log_loss': log_loss(labels, probs, labels=classes)}

    args = TrainingArguments(
        output_dir='/kaggle/working/out_lora',
        per_device_train_batch_size=8,
        per_device_eval_batch_size=16,
        num_train_epochs=1,                         # keep light for Kaggle
        gradient_accumulation_steps=2,
        learning_rate=2e-4,
        evaluation_strategy='steps', eval_steps=200,
        logging_steps=100,
        save_strategy='no',
        fp16=torch.cuda.is_available(),
        report_to=[]
    )

    trainer = Trainer(model=model, args=args, train_dataset=ds_tr, eval_dataset=ds_va, compute_metrics=compute_metrics)
    trainer.train()

    # Temperature fit on val for mild calibration (offline)
    with torch.no_grad():
        va_logits = torch.tensor(trainer.predict(ds_va).predictions, device=device)
        va_labels = torch.tensor(y[va_idx], device=device)

    temperature = torch.nn.Parameter(torch.ones((), device=device))
    opt = torch.optim.LBFGS([temperature], lr=0.1, max_iter=50)
    def nll_with_temperature():
        opt.zero_grad()
        scaled = va_logits / temperature.clamp_min(1e-3)
        loss = torch.nn.functional.cross_entropy(scaled, va_labels)
        loss.backward(); return loss
    opt.step(nll_with_temperature)
    T = float(temperature.detach().cpu().numpy())
    print(f'[LoRA] fitted temperature: T={T:.3f}')

    # Build pseudo-OOF (for ensemble weight search) and test probabilities
    with torch.no_grad():
        tr_logits = torch.tensor(trainer.predict(ds_tr).predictions, device=device)
        tr_probs  = torch.softmax(tr_logits / T, dim=1).cpu().numpy()
        va_probs  = torch.softmax(va_logits / T, dim=1).cpu().numpy()
    lora_oof = np.zeros((len(train_df), 3), dtype=float)
    lora_oof[tr_idx] = tr_probs; lora_oof[va_idx] = va_probs
    print('[LoRA] pseudo-OOF log_loss:', log_loss(y, lora_oof, labels=classes))

    with torch.no_grad():
        test_logits = torch.tensor(trainer.predict(tokenized_test).predictions, device=device)
        lora_test_proba = torch.softmax(test_logits / T, dim=1).cpu().numpy()
else:
    print("MODE does not include embeddings_lora → skipping.")


# 8) ENSEMBLE (only if MODE=="ensemble")

In [10]:
if MODE == "ensemble":
    sources = []
    tests   = []
    if lex_best_oof is not None:
        sources.append(("lex", lex_best_oof))
        tests.append(("lex", predict_from_folds(lex_best_models, lex_best_scalers, X_lex_test.values.astype(float))))
    if emb_best_oof is not None and X_emb_test is not None:
        sources.append(("emb", emb_best_oof))
        tests.append(("emb", predict_from_folds(emb_best_models, emb_best_scalers, X_emb_test)))
    if lora_oof is not None and lora_test_proba is not None:
        sources.append(("lora", lora_oof))
        tests.append(("lora", lora_test_proba))

    if len(sources) == 0:
        raise RuntimeError("No component models available to ensemble. Run MODE='lexical' or 'embeddings' first.")

    # Two-source blend -> 1D grid search; 3+ sources -> simple stacker
    if len(sources) == 1:
        name, oof = sources[0]
        test = dict(tests)[name]
        final_test = test
        print(f"Only one source ({name}); using it as ensemble output.")
    elif len(sources) == 2:
        (n1, o1), (n2, o2) = sources
        (t1_name, t1), (t2_name, t2) = tests
        assert n1==t1_name and n2==t2_name
        w, best = best_weight_for_blend(y, o1, o2)
        print(f"Best blend weight for {n1}: {w:.2f} (OOF logloss={best:.5f})")
        final_test = w*t1 + (1-w)*t2
    else:
        # Tiny stacker (multinomial LR) on concatenated OOF probs
        from sklearn.linear_model import LogisticRegression
        names = [n for n,_ in sources]
        OOF   = np.hstack([p for _,p in sources])
        meta  = LogisticRegression(max_iter=1000, multi_class='multinomial', random_state=SEED)
        meta.fit(OOF, y)
        TEST  = np.hstack([dict(tests)[n] for n in names])
        final_test = meta.predict_proba(TEST)
        print("Stacker trained over sources:", names)

    # Optional: temperature-scale the ensemble using best available OOF proxy (concat if needed)
    try:
        if len(sources) >= 1:
            if len(sources) == 1:
                oof_ref = sources[0][1]
            else:
                # average OOF as a rough proxy for calibration
                oof_ref = np.mean([p for _,p in sources], axis=0)
            T = fit_temperature(oof_ref, y)
            final_test = apply_temperature(final_test, T)
            print(f"Applied temperature scaling: T={T:.3f}")
    except Exception as e:
        print("Temp scaling skipped:", e)

    sub_blend = pd.DataFrame({
    'id': test_df['id'].values,
    'winner_model_a': final_test[:,0],
    'winner_model_b': final_test[:,1],
    'winner_tie':     final_test[:,2],
    })

Best blend weight for lex: 0.45 (OOF logloss=1.05122)
Applied temperature scaling: T=0.688


In [ ]:
chosen = None

if MODE == "lexical":
    assert 'lex_proba_test' in globals(), "Run lexical block first."
    chosen = lex_proba_test

elif MODE == "embeddings":
    assert 'emb_proba_test' in globals(), "Run embeddings block first."
    chosen = emb_proba_test

elif MODE == "embeddings_lora":
    assert 'lora_test_proba' in globals(), "Run embeddings_lora block first."
    chosen = lora_test_proba

elif MODE == "ensemble":
    assert 'final_test' in globals(), "Run ensemble block to produce final_test."
    chosen = final_test

else:
    raise ValueError(f"Unknown MODE: {MODE}")

✅ Wrote submission.csv using MODE="ensemble"


In [ ]:
# Save OOF predictions to numpy files
np.save('/kaggle/working/lex_best_oof.npy', lex_best_oof)
np.save('/kaggle/working/emb_best_oof.npy', emb_best_oof)
np.save('/kaggle/working/y_true.npy', y)


# Save X_lex for structural feature analysis
X_lex.to_csv('/kaggle/working/X_lex_train.csv', index=False)